In [1]:
import torch
import torchvision
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.transforms import v2 as v2
import os.path as osp
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [3]:
size = (100*100)

transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((100, 100)),
    torchvision.transforms.RandomHorizontalFlip(p=0.5),
    torchvision.transforms.RandomRotation(15),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = torchvision.datasets.ImageFolder(root='/home/student/data/DL_Bootcamp2025_HW2/train', transform=transform)
test_dataset = torchvision.datasets.ImageFolder(root='/home/student/data/DL_Bootcamp2025_HW2/test', transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

print(len(train_dataset))
print(len(test_dataset))
print(train_dataset[0][0].shape)

18000
2000
torch.Size([3, 100, 100])


In [4]:
class ResidualBlock(nn.Module):
    def __init__(
        self, in_channels: int, out_channels: int
    ):
        super().__init__()
        stride: int = 1
        self.conv1 = nn.Conv2d(in_channels, out_channels, 3, stride, 1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, 3, stride, 1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.act = nn.GELU()
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        identity = x 
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.act(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += identity
        out = self.act(out)
        return out
    
class DownsamplingResidualBlock(nn.Module):
    def __init__(
        self, in_channels: int, out_channels: int
    ):
        super().__init__()
        stride: int = 2
        self.conv1 = nn.Conv2d(in_channels, out_channels, 3, stride, 1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.act = nn.GELU()
        self.conv2 = nn.Conv2d(out_channels, out_channels, 3, 1, 1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.downsample = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
            nn.BatchNorm2d(out_channels),
        )

    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        identity = self.downsample(x)  # Skip connection -> 전달 해줄 거 미리 저장해놓고
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.act(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += identity # skip connection -> 전달
        out = self.act(out)
        return out
        
class ResNet34(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False),
            nn.BatchNorm2d(64),
            nn.GELU(),
        )
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1) 
        
        self.stage_1 = nn.Sequential(
            ResidualBlock(64, 64), # in/ out 채널 수
            ResidualBlock(64, 64),
            ResidualBlock(64, 64),
            torch.nn.Dropout(0.5)
        )
        self.stage_2 = nn.Sequential(
            DownsamplingResidualBlock(64, 128),
            ResidualBlock(128, 128),
            ResidualBlock(128, 128),
            ResidualBlock(128, 128),
            torch.nn.Dropout(0.5)
        )
        self.stage_3 = nn.Sequential(
            DownsamplingResidualBlock(128, 256),
            ResidualBlock(256, 256),
            ResidualBlock(256, 256),
            ResidualBlock(256, 256),
            ResidualBlock(256, 256),
            torch.nn.Dropout(0.5)
        )
        
        self.classifier = nn.Linear(256, 512)
        torch.nn.Dropout(0.5)
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        feature = self.conv_1(x)
        feature = self.maxpool(feature)
        feature = self.stage_1(feature)
        feature = self.stage_2(feature)
        feature = self.stage_3(feature)
        feature_vector = torch.mean(feature, dim=(2, 3))
        class_logit = self.classifier(feature_vector)
        return class_logit

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ResNet34()
model  = model.to(device)
criterion = nn.CrossEntropyLoss()  
optimizer = optim.SGD(model.parameters(), momentum=0.9, lr=0.01, nesterov=True)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)  

def train(model, train_loader, criterion, optimizer, scheduler, epochs=50):
    model.train()
    for epoch in range(epochs):
        total_loss = 0.0
        for images, labels in train_loader:
            images = images.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            outputs = model.forward(images)
            #y_hat = softmax(logits)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        scheduler.step() 
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss / len(train_loader):.4f}, LR: {scheduler.get_last_lr()[0]:.5f}")

In [6]:
def test(model, test_loader):
    model.eval()
    correct, total = 0, 0
    preds = []
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)

            preds += predicted.detach().cpu().numpy().tolist()
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    import pandas as pd
    df = pd.read_csv('./hw2_sample_submission.csv')
    df['Label'] = preds
    df.to_csv('./submission.csv', index=False)

    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")

In [7]:
import os, random, torch
import numpy as np

def set_seed(seed):
    os.environ['PYTHONHASHSEED'] = str(seed) 
    random.seed(seed)  
    np.random.seed(seed)  
    torch.manual_seed(seed)  
    torch.cuda.manual_seed(seed)  
    torch.cuda.manual_seed_all(seed)  
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(123)

In [8]:
train(model, train_loader, criterion, optimizer, scheduler, epochs=50)
test(model, test_loader)

Epoch [1/50], Loss: 2.4706, LR: 0.01000
Epoch [2/50], Loss: 2.0131, LR: 0.01000
Epoch [3/50], Loss: 1.7952, LR: 0.01000
Epoch [4/50], Loss: 1.6289, LR: 0.01000
Epoch [5/50], Loss: 1.5201, LR: 0.01000
Epoch [6/50], Loss: 1.4311, LR: 0.01000
Epoch [7/50], Loss: 1.3408, LR: 0.01000
Epoch [8/50], Loss: 1.2775, LR: 0.01000
Epoch [9/50], Loss: 1.2240, LR: 0.01000
Epoch [10/50], Loss: 1.1539, LR: 0.00500
Epoch [11/50], Loss: 0.9967, LR: 0.00500
Epoch [12/50], Loss: 0.9542, LR: 0.00500
Epoch [13/50], Loss: 0.9223, LR: 0.00500
Epoch [14/50], Loss: 0.8939, LR: 0.00500
Epoch [15/50], Loss: 0.8691, LR: 0.00500
Epoch [16/50], Loss: 0.8434, LR: 0.00500
Epoch [17/50], Loss: 0.8126, LR: 0.00500
Epoch [18/50], Loss: 0.7930, LR: 0.00500
Epoch [19/50], Loss: 0.7762, LR: 0.00500
Epoch [20/50], Loss: 0.7532, LR: 0.00250
Epoch [21/50], Loss: 0.6372, LR: 0.00250
Epoch [22/50], Loss: 0.6052, LR: 0.00250
Epoch [23/50], Loss: 0.5914, LR: 0.00250
Epoch [24/50], Loss: 0.5710, LR: 0.00250
Epoch [25/50], Loss: 0.55